# ZRE Projekt - 2023
#### Honza Černocký a Honza Pavlus, FIT VUT Brno, 2023
#### ` Celkem bodů 29`
---

Principem projektu oprava signálů narušených klipováním pomocí lineární predikce nebo jiných technik. Nachystali jsme 100 signálů z TIMITu tak, že máte k disposici originál i klipovanou verzi. Klipovaná verze simuluje ztrátu nejvyššího bitu. Signály a seznam viz https://www.fit.vutbr.cz/study/courses/ZRE/public/2022-23/. 

Projekt doporučujeme řešit v Pythonu v Google Colab, ale je možné použít i jiný jazyk / prostředí, v tomto případě prosím zašlete protokol a zdrojáky emailem.  

---



In [ ]:
# nacteni signalu a listu, at to nemusite hledat
!wget https://www.fit.vutbr.cz/study/courses/ZRE/public/2022-23/proj_zre_2023.tgz   
!tar xfz proj_zre_2023.tgz
!ls

--2023-03-30 12:25:48--  https://www.fit.vutbr.cz/study/courses/ZRE/public/2022-23/proj_zre_2023.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12170967 (12M) [application/x-gtar]
Saving to: ‘proj_zre_2023.tgz’

proj_zre_2023.tgz   100%[===================>]  11.61M   272KB/s    in 49s     

2023-03-30 12:26:40 (240 KB/s) - ‘proj_zre_2023.tgz’ saved [12170967/12170967]

clipped_signals    orig_signals       sample_data
list_students.txt  proj_zre_2023.tgz


## Úloha 1 - Detektor klipovaných signálů - `2 body `

Navrhněte a naprogramujte detektor klipovaných částí signálu. Pozor, klipované mohou být jak kladné, tak záporné vzorky. Zobrazte několik příkladů těchto klipovaných částí v časové a ve frekvenční oblasti (spektrogram). 

---

### Vaše řešení:

Popište vaše řešení

In [ ]:
# Místo pro kód

## Úloha 2 - Výpočet SNR - `2 body `

Napište funkci pro výpočet poměru signálu k šumu, kde šum (jmenovatel) bude chyba (rozdíl mezi klipovaným signálem a originálem) a signál (čitatel) je originální signál. Spočítejte SNR způsobené klipováním pro všech 100 signálů. Berte je v úvahu celé, nejen klipované části.

---


### Vaše řešení:

Popište vaše řešení

In [ ]:
# Místo pro kód

## Úloha 3 - Oprava pomocí predikce zleva - ` 5 bodů`

Napište funkci, která bude opravovat klipované části. Měla by pracovat tak, že na L=240 vzorcích signálu před klipovanou částí odhadne koeficienty prediktoru a ten využije k náhradě klipovaných vzorků predikcí z minulosti. Vyberte 2 opravené segmenty a zobrazte je. Připravte dvě promluvy k přehrání (pokud budete pracovat v Py notebooku, přímo v něm, pokud v něčem jiném, jako několik WAV souborů). Vyzkoušejte délky prediktoru od P=2 do P=20. Pro každou délku vyhodnoťte na všech 100 signálech SNR a spočítejte redukci SNR oproti původnímu stavu (SNRR - v procentech). 

---


### Vaše řešení:

Popište vaše řešení

In [ ]:
# Místo pro kód

## Úloha 4 - Oprava pomocí predikce zprava - ` 4 body`

Zopakujte předchozí cvičení, ale tentokrát predikujte klipované vzorky zprava (tedy z budoucnosti). Ve výpočtech budete tedy muset obracet signály nebo použít nekauzální filtry. Udělejte to samé, co v minulé cvičení, tedy vzorky k zobrazení a poslechu, závislost SNR a SNRR na délce prediktoru.

### Vaše řešení:

Popište vaše řešení

In [ ]:
# Místo pro kód

## Úloha 5 - Lineární kombinace obou predikcí - ` 4 body`

Pro klipované vzorky proveďte lineární kombinaci obou predikcí, můžete vyzkoušet různé interpolace (s konstantními váhami 0,5 a 0,5, s klesajícím oknem zleva a stoupajícím zprava, cokoliv jiného). Proveďte opět ty samé kroky jako v minulých cvičeních, tedy vzorky k zobrazení a poslechu, závislost SNR a SNRR na délce prediktoru. 

---


### Vaše řešení:

Popište vaše řešení

In [ ]:
# Místo pro kód

## Úloha 6 - Cokoliv lepšího - ` 12 bodů`

Navrhněte a implementujte jakékoliv zlepšení rekonstrukce, možné příklady jsou: 

*   Optimalizace délky okna. 
*   Korektní práce s minulými či budoucími vzorky, pokud je za sebou více klipovaných částí (neměly by vstupovat do korelace pro výpočet R[k] a už vůbec ne do predikce vzorků)
*   Využití dynamické délky okna a dynamické délky prediktoru podle charakteru signálu. 
* Dlouhodobá predikce využívající základního tónu. 
* Analytické odvození, jak má predikce vzorků probíhat simultánně z levého či pravého kontextu nebo nalezení v literatuře a implementace. 
* Využití strojového učení (např. neuronová síť predikující vzorky)
* Cokoliv jiného  

Pro navrženou metodu proveďte opět ty samé kroky jako v minulých cvičeních, tedy vzorky k zobrazení a poslechu, a výpočet závislost SNR a SNRR. Komentujte výsledky. **Toto cvičení se bude obhajovat v pondělí 15. května**, nachystajte si prosím max. 10-minutovou presentaci - pár slajdů s popisem, co jste dělali, SNRR, ploty. 

---





### Vaše řešení:

Popište vaše řešení

In [ ]:
# Místo pro kód

# Hodnocení

Viz zadání. Jak je v ZRE zvykem, student/ka s nejinovativnějším / nejzajímavějším řešením dostane láhev dobrého červeného.

# Speciální projekty

Pokud máte zájem o zpracován jiného zadání (např. rozpoznávání řeči, jazyka, mluvčího), nebo na něčem děláte v rámci BP / DP / projektové praxe / ve firmě, atd., prosím kontaktujte mě co nejdříve. Co nejdříve se osobně domluvíme, zadání musím předem odsouhlasit.